In [1]:
import numpy as np
import torch.nn as nn
import torch
import torch.optim as optim
from torchvision import datasets, transforms
import emnist
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
import loader
mnist_images, mnist_labels = emnist.extract_test_samples('digits')

train_dataset = datasets.MNIST(root='./mnist_data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='./mnist_data/',
                              train=False,
                              transform=transforms.ToTensor(),
                              download=True)

# animals_dataset = loader.load_chapel_dataset('./animals-10')

In [2]:
model = nn.Sequential(
    nn.Conv2d(in_channels=1,out_channels=12,kernel_size=3,stride=2),
    nn.Conv2d(in_channels=12,out_channels=16,kernel_size=4,stride=1),
    nn.MaxPool2d(kernel_size=2),
    nn.Flatten(),
    nn.LazyLinear(out_features=10),
    nn.Softmax(dim=1)
)

/opt/homebrew/lib/python3.11/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [3]:
model

Sequential(
  (0): Conv2d(1, 12, kernel_size=(3, 3), stride=(2, 2))
  (1): Conv2d(12, 16, kernel_size=(4, 4), stride=(1, 1))
  (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): Flatten(start_dim=1, end_dim=-1)
  (4): LazyLinear(in_features=0, out_features=10, bias=True)
  (5): Softmax(dim=1)
)

In [4]:
img = mnist_images[0,:,:]
img = torch.from_numpy(img.reshape(1,1,28,28)).float()
img.shape

/var/folders/wt/nr12w9r958d0nwkk86zwr9600000gp/T/ipykernel_17231/3604749479.py:2: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_numpy.cpp:212.)
  img = torch.from_numpy(img.reshape(1,1,28,28)).float()


torch.Size([1, 1, 28, 28])

In [5]:
model(img).shape
model(img)

tensor([[1.4505e-18, 1.5648e-24, 8.6627e-20, 9.9598e-01, 2.8072e-10, 1.2934e-05,
         1.3050e-20, 2.1308e-08, 4.0080e-03, 4.8275e-17]],
       grad_fn=<SoftmaxBackward0>)

In [6]:
model = nn.Sequential(
    nn.Conv2d(in_channels=1,out_channels=12,kernel_size=3,stride=2),
    nn.Conv2d(in_channels=12,out_channels=16,kernel_size=4,stride=1),
    nn.MaxPool2d(kernel_size=2),
    nn.Flatten(),
    nn.LazyLinear(out_features=10),
    nn.Softmax(dim=1)
)

epochs = 5
learning_rate = 0.05
batch_size = 30

loss_criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

model.train()

for epoch in range(epochs):
    epoch_loss = 0
    num_pairs = len(train_dataset)# mnist_images.shape[0]
    num_batches = num_pairs // batch_size
    with tqdm(total=num_batches) as pbar:
        for batch, labels in train_loader:
            # for b in range(num_batches):
            # batch = mnist_images[b*batch_size:(b+1)*batch_size,:,:]
            # batch = torch.from_numpy(batch.reshape(batch_size,1,28,28)).float() / 255
            # labels = torch.from_numpy(mnist_labels[b*batch_size:(b+1)*batch_size])

            optimizer.zero_grad()
            output = model(batch)
            loss = loss_criterion(output,labels)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
            pbar.update(1)
            # print(loss.item())
        # for (im_idx, label) in zip(range(num_pairs), mnist_labels):
        #     im = mnist_images[im_idx,:,:]
        #     im = torch.from_numpy(im.reshape(1,1,28,28)).float()

        #     optimizer.zero_grad()
        #     output = model(im)
        #     label_vector = torch.zeros(1,10)
        #     label_vector[0,label] = 1
        #     loss = loss_fn(output,label_vector)# -torch.sum(torch.log(output))
        #     loss.backward()
        #     optimizer.step()
        #     # print(loss.item(),output[0,label].item())
        #     epoch_loss += loss.item() 
        #     pbar.update(1)
    print('loss: ', epoch_loss / num_pairs)
    

/var/folders/wt/nr12w9r958d0nwkk86zwr9600000gp/T/ipykernel_17231/1003746249.py:27: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  with tqdm(total=num_batches) as pbar:


  0%|          | 0/2000 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [137]:
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

model.eval()
correct = 0
for data,target in test_loader:
    output = model(data)
    pred = output.data.max(1, keepdim=True)[1]
    correct += pred.eq(target.data.view_as(pred)).cpu().sum().item()
print('accuracy: ', correct ,'/', len(test_dataset))

accuracy:  9723 / 10000


In [109]:
(im,lb) = next(iter(train_loader))
lb.shape

torch.Size([10])

In [112]:
transforms.ToTensor()

ToTensor()